# **This code converts 'model_v15' into TFlite model and then into .h file without quantization  DM2, D4**<br>
Author: Jatin Kadge<br>
Roll No.: EE23D021 <br>
Date: 05 Dec 2023

In [ ]:
%tensorflow_version 2.1

#%tensorflow_version` only switches the major version: 1.x or 2.x.
#You set: `2.1`. This will be interpreted as: `2.x`.

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
from tensorflow.keras import layers

In [ ]:
# Print versions
!python --versions
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
#print('keras ' + tf.keras.version.VERSION)

unknown option --versions
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
Numpy 1.23.5
TensorFlow 2.14.0


In [ ]:
tflite_model_name = 'tflite_dmodel__v10_4_DM2_3_3secs_2'  # Will be given .tflite suffix
c_model_name = 'c_dmodel__v15_DM2_3_3secs_2'       # Will be given .h suffix


In [ ]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
  #Load the Saved model
  loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/iGest/Data/05_Dec/Models/dmodel_v15-DM2-3_3secs_reduced_2.keras")

  #Drive Link: https://drive.google.com/drive/folders/19_DLihXj594NRIow7G-10f4maR_aOd3D?usp=sharing

In [ ]:
# View model
loaded_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 48, 6, 4)          16        
                                                                 
 conv2d_19 (Conv2D)          (None, 46, 6, 4)          52        
                                                                 
 dropout_9 (Dropout)         (None, 46, 6, 4)          0         
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 23, 6, 4)          0         
 g2D)                                                            
                                                                 
 flatten_9 (Flatten)         (None, 552)               0         
                                                                 
 dense_18 (Dense)            (None, 10)                5530      
                                                      

In [ ]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = []
converter.allow_custom_ops = False
converter.experimental_new_converter = True
tflite_model = converter.convert()
#droping default optimisation function and custom all ops function above

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

import os
basic_model_size = os.path.getsize("tflite_dmodel__v15_DM2_3_3secs_2.tflite")
print("Model is %d bytes" % basic_model_size)

Model is 25964 bytes


In [ ]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [ ]:
# Write TFLite model to a C source (or header) file
with open(c_model_name + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, c_model_name))